# This notebook loads camera and treadmill clock signals and saves start times in .evt.tre and .evt.cam files

### *.dat* file is needed to extract clock signals
### *.prm* file is needed to load the sampling frequency
### behavior data (*.behav_param* and *.entrancetimes*) needed to separate treadmill recording from homecage recording.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, gc
import glob
from IPython.display import clear_output
from pprint import pprint  #if the module is not available, comment this line out
from shutil import copy,rmtree

#utility function to check if a given signal is a binary signal using
#correlation coeficient
def isClock(sig,fs):
    sigM=np.mean(sig)
    sig2=sig-sigM
    sig2=np.sign(sig2)#clock is a perfect binary signal now
    if len(sig) > 5*60*fs: #5min
        L=int(5*60*fs)
    else:
        L=len(sig)-1
    if np.corrcoef(sig[:L],sig2[:L])[0,1] > 0.98:
        return True
    else:
        return False

if "__file__" not in dir():
    %run UtilityTools.ipynb


In [ ]:
class ephy_events:
    def __init__(self,sessionPath,nbTre,nbCam,overwrite=False):
        '''
        loads required information from sessionPath and writes
        the *.evt files for each session
        '''
        self.sessionPath=sessionPath
        self.nbTre=nbTre
        self.nbCam=nbCam
        self.overwrite=overwrite
        
    def run(self):
        if self._check_input():        
            #reading the prm file to get the required info
            nameDic=prm_reader(self.prmFile)
            fs=nameDic['sample_rate']
            nchannels=nameDic['nchannels']
            del nameDic

            #reading the dat file to get the required info
            dat=np.fromfile(self.datFile,dtype=np.int16)
            dat=np.reshape(dat,(-1,nchannels),order='C')
            treClock=dat[:,-abs(self.nbTre)].copy()
            camClock=dat[:,-abs(self.nbCam)].copy()
            del dat

            treRate=self.clock_to_rate(treClock,fs)
            camRate=self.camera_clock_to_rate(camClock,fs)
            del treClock,camClock
            treEventResult=self.write_evt_file(treRate,fs)
            camEventResult=self.write_evt_file(camRate,fs)
            del camRate,treRate
            self.result=treEventResult and camEventResult

            gc.collect()
        else:
            self.result=False
    
    def _check_input(self):
        behavFile=find_file(self.sessionPath,['.behav_param'])[0]
        etFile=find_file(self.sessionPath,['.entrancetimes'])[0]
        self.datFile=find_file(self.sessionPath,['.dat'])[0]
        self.prmFile=find_file(self.sessionPath,['.prm'])[0]
        self.experiment=os.path.basename(self.prmFile)[:-4]

        if (not os.path.exists(behavFile)) or (not os.path.exists(etFile)):
            return []
        if (not os.path.exists(self.datFile)) or (not os.path.exists(self.prmFile)):
            return []
        if self.overwrite is False:
            cond1=os.path.exists(find_file(self.sessionPath,['.evt.cam'])[0])
            cond2=os.path.exists(find_file(self.sessionPath,['.evt.tre'])[0])
            if cond1 and cond2:
                return None
        return True

    def clock_to_rate(self,clock,fs):
        '''
        this function recieves a clock signal and returns correspondant
        rate signals with 2 levels:
        0: no pulses
        1: pulses
        '''
        if isClock(clock,fs) == False:
            return False
        clockM=np.mean(clock)
        clock=clock-clockM
        clock=np.sign(clock)#clock is a perfect binary signal now
        frameShotTime=np.diff(clock)
        frameShotTime[frameShotTime<0]=0 #a single pulse on the rising edge
        shotTimeIndex=np.nonzero(frameShotTime>0)[0]

        shotDistance=np.diff(shotTimeIndex)
        interTrialOnset=np.nonzero(shotDistance>2*fs)[0]
        frameRate=np.ones(clock.shape)
        for i in range(len(interTrialOnset)):
            frameRate[shotTimeIndex[interTrialOnset[i]]:
                      shotTimeIndex[interTrialOnset[i]+1]]=0
        frameRate[0:shotTimeIndex[0]]=0
        frameRate[shotTimeIndex[-1]:]=0
        return frameRate
    
    def camera_clock_to_rate(self,clock,fs):
        '''
        this function recieves a clock signal and returns correspondant
        rate signals with 3 levels:
        0: no pulses
        1: slow pulses
        2: fast pulses
        '''
        if isClock(clock,fs) == False:
            return False
        clockM=np.mean(clock)
        clock=clock-clockM
        clock=np.sign(clock)#clock is a perfect binary signal now
        frameShotTime=np.diff(clock)
        frameShotTime[frameShotTime<0]=0 #a single pulse on the rising edge
        frameRateIndex=np.reciprocal(np.diff(np.nonzero(frameShotTime>0.5)[0])/fs)
        frameRateIndex=frameRateIndex-np.mean(frameRateIndex)
        frameRateIndex=(np.sign(frameRateIndex)+3)/2 #to map to 1&2
        timeFrameRate=np.zeros(clock.shape)
        timeFrameRate[np.nonzero(frameShotTime>0.5)[0]]=np.append(frameRateIndex,frameRateIndex[-1])
        frameType={
            0: np.nonzero(timeFrameRate==1)[0], #slow
            1: np.nonzero(timeFrameRate==2)[0] #fast
            }
        index=np.min([frameType[0][0],frameType[1][0]])
        firstFrameType=np.argmin([frameType[0][0],frameType[1][0]])
        frameRate=np.zeros(timeFrameRate.shape)
        while True:  #the loop iterates over trials not data samples
            frameRate[index:]=firstFrameType+1 #convert to 1 or 2
            firstFrameType = not firstFrameType
            tmpIndex=np.nonzero(frameType[firstFrameType]>index)[0]
            if tmpIndex.size>0:
                index=frameType[firstFrameType][tmpIndex[0]]
            else:
                break
        index=np.max([frameType[0][-1],frameType[1][-1]])
        frameRate[index:]=0
        return frameRate

    def write_evt_file(self,sig,fs):
        uniqueValues=np.unique(sig)
        fileType=''
        if len(uniqueValues)==2:
            fileType='.evt.tre'
        elif len(uniqueValues)==3:
            fileType='.evt.cam'
        output=os.path.join(self.sessionPath,self.experiment+fileType)
        if os.path.exists(output) and not self.overwrite:
            return None
        if len(uniqueValues)>3 or len(uniqueValues)<2:
            #0,1 and 2 in case of camera
            print("bad clock signal, cannot write:",output)
            return False

        clock=sig.copy()
        clock[clock<uniqueValues[-1]]=0
        clock[np.append(0,np.diff(clock))<=0]=0   #each nonzero element is an event
        events=np.nonzero(clock)[0]
        eventList_ms=[events[i]/(fs/1000) for i,_ in enumerate(events)]
        try:
            with open(output,'w') as f:
                for event in eventList_ms:
                    f.write(str(event))
                    f.write('\n')   #new line
        except Exception as e:
            print(repr(e))
            return False
        print("wrote:",output)
        return True


In [ ]:
class ephy_event_batch(ephy_events):
    def __init__(self,root, animalList, nbTre, nbCam, overwrite):
        self.failedSessions={'Write':[],'Exist':[],'NoBehavior':[]}
        for animal in animalList:
            animalPath=os.path.join(root,animal)
            sessionList=[os.path.basename(expPath) for expPath in 
                         glob.glob(os.path.join(animalPath,"Experiments",animal+'*'))]
            sessionList=sorted(sessionList)
            for session in sessionList:
                sessionPath=os.path.join(animalPath,"Experiments",session)
                super().__init__(sessionPath,nbTre,nbCam,overwrite=overwrite).run()
                if self.result is None:
                    self.failedSessions['Exist'].append(session)
                elif self.result==[]:
                    self.failedSessions['NoBehavior'].append(session)
                elif self.result is False:
                    self.failedSessions['Write'].append(session)
        clear_output()
    def __repr__(self):
        print('\nfailed sessions:')
        try:
            pprint(self.failedSessions)
        except:
            print(self.failedSessions)
        return ''

## Script to Run the notebook as a Batch
### only sessions with *.dat*, *.prm*, *.behav_param* and *.entrancetimes* files will be included
#### root: path to main data folder
#### animalList: animals for which to generate the *.evt* files
#### nbTre: channel number for TREADMILL signal with reference to the last channel. ex: -1(last channel), -2(channel before the last one)
#### nbCam: channel number for CAMERA signal with reference to the last channel. ex: -1(last channel), -2(channel before the last one

In [ ]:
if "__file__" not in dir():
    root="/data/"
    #in Windows paths must be like this: root="C:\\Data\\Recordings\\" (double backslash instead of single)
    
    animalList=["Rat173"]
    
    nbTre=-2
    
    nbCam=-1

    overwrite=False
  
    #--------------------------------------------------------------------------------
    a=ephy_event_batch(root, animalList, nbTre, nbCam, overwrite)
    a

# Script to save EVT files on NAS as a batch

In [ ]:
def run_event_batch(root,animalList,nbTre,nbCam,fileTypes=['.dat','.prm','.behav_param','.entrancetimes'],overwrite=False):
    failedSessions={'Exist':[],'NoBehavior':[],'Write':[]}
    for animal in animalList:
        animalFolder=os.path.join(root,animal,'Experiments')
        sessionSets=[set([os.path.dirname(i) for i in find_file(animalFolder,[j])]) for j in fileTypes]
        goodSessions=set.intersection(*sessionSets)
        goodSessions=sorted(list(goodSessions))
        
        for session in goodSessions:
            safeFiles=[safe_copy_from_nas(os.path.join( session,os.path.basename(session) )+f) for f in fileTypes]
            try:
                newFilePaths=[safeFile.start('.NAS4EVTqwrytdvnz2u1r') for safeFile in safeFiles]
                obj=ephy_events(os.path.dirname(newFilePaths[0]),nbTre,nbCam,overwrite=overwrite)
                obj.run()
                res=obj.result
                if res is None:
                    failedSessions['Exist'].append(session)
                elif res==[]:
                    failedSessions['NoBehavior'].append(session)
                elif res is False:
                    failedSessions['Write'].append(session)            
            except Exception as e:
                print('Event file creation failed at:',session)
                print(repr(e))
            finally:
                for safeFile in safeFiles:
                    safeFile.stop(fileTypes=['.evt.cam','.evt.tre'])

    return failedSessions

In [ ]:
if "__file__" not in dir():
    root="/NAS02"
    
    animalList=["Rat173"]
    
    nbTre=-2
    
    nbCam=-1

    overwrite=False
  
    res=run_event_batch(root,animalList,nbTre,nbCam,overwrite=False)
    res